## Subject-Verb-Objects

In this notebook, we conduct a series of experiments in order, first, toisolate the subject-verb-object (SVO) triples in the texts of speakers we have gendered male or female, then to explore the *character spaces* they establish for gendered entities within their speech as well as the nature of the *character space* they create for themselves as speakers. 

**Note**: We are not excluding parentheticals in this notebook.

**Next Steps**: Work on code to compile / visualize this as a network graph (?).

## Load Libraries & Data

In [1]:
# IMPORTS
import re, spacy, textacy
import pandas as pd
from nltk import sent_tokenize

# Loading the Data in a gendered partitioned fashion: 
talks_m = pd.read_csv('talks_male.csv', index_col='Talk_ID')
talks_f = pd.read_csv('talks_female.csv', index_col='Talk_ID')
talks_nog = pd.read_csv('talks_nog.csv', index_col='Talk_ID')
talks_all = pd.concat([talks_m, talks_f, talks_nog])

# And then grabbing on the texts of the talks:
texts_all = talks_all.text.tolist()
texts_women = talks_f.text.tolist()
texts_men = talks_m.text.tolist()

print(f"From our {talks_all.shape[0]}x{talks_all.shape[1]} CSV, \
we have a list of {len(texts_all)} talks: {len(texts_women)} by women and \
{len(texts_men)} by men.")

From our 992x14 CSV, we have a list of 992 talks: 260 by women and 720 by men.


Lowercasing everything upfront because we don't care whether it is *She* or *she*. 

In [2]:
# Lowercase everything before we create spaCy doc and Textacy SVO triple
texts_w = [text.lower() for text in texts_women]
texts_m = [text.lower() for text in texts_men]

spaCy has three different English language models: small, medium, and large. We use the large model here because our corpus is small and the syntax may be a bit more involved. 

`TO DO`: determine the difference between the models. 

After determining telling spaCy which model to use, we then use its conventions for feeding a set of texts as a list of strings, to it. 

The preview simply checks that everything went as planned: it gives us a word count and the first 50 characters -- which is weird because in theory it has converted the string to a series of spacy objects. 

In [3]:
# Load the Space pipeline to be used
nlp = spacy.load('en_core_web_lg')

# Use the pipe method to feed documents 
docs_w = list(nlp.pipe(texts_w))
docs_m = list(nlp.pipe(texts_m))

# A quick check of our work:
docs_m[0]._.preview

'Doc(2690 tokens: "  thank you so much, chris. and it\'s truly a gr...")'

## SVO Dataframes for Male and Female Speakers

We now have two sets of spaCy documents, one for the women speakers and one for the men speakers -- there is a third set, composed of groups of speakes, but we are leaving them aside for now. 

In order to examine how each set of texts imagines the agency and actions of gendered persons in their speech, we are going to go through all the clauses in each talk, identify those clauses that have a pronoun as a subject. We are going to compile all the clauses that feature all the subject case pronouns: *first person* (I and we), *second person* (you), and *third person* (she, he, they, it).

`TO DO`: Identify the clauses where gendered entities are acted upon: her, him, me, us, them. (Convert the **spaCy object** in the function below to strings so that we can then match for the objective pronouns?) This may or may not be possibly best done as a separate section.

In the next two cells we create the list of pronouns for whom we seek SVO triples and then we have the function that will grab SVOs starting with those pronouncs and append them to a list each item of which is a dictionary. Each dictionary consists of three entries: one for the subject, one for the verb, and one for the object. This structure was chosen because it makes it easier to instantiate a pandas dataframe in the work that follows.

In [11]:
# Create lists of gendered words

subjects = 'i she he it you man woman'

objects = 'him her'

gendered = subjects.split(' ')

print(gendered)

['i', 'she', 'he', 'it', 'you', 'man', 'woman']


In [12]:
def svos(doc, svo_list):
    svotriples = list(textacy.extract.triples.subject_verb_object_triples(doc))
    for item in svotriples:
        svo_list.append(
            {
                'subject': str(item[0][-1]), 
                'verb': str(item[1][-1]), 
                'object': str(item[2])
            }
        )

In [14]:
# Create the two lists
svolist_m = []
svolist_w = []

# Populate the lists with SVO triples
for doc in docs_m:
    svos(doc, svolist_m)

for doc in docs_w:
    svos(doc, svolist_w)

# Convert the lists to dataframes
svos_w = pd.DataFrame(svolist_w)
svos_m = pd.DataFrame(svolist_m)

print(df_all_svos_m.shape, df_all_svos_w.shape)

(80331, 3) (26527, 3)


# Counting Verbs

In [15]:
test = svos_w[svos_w["subject"] == "he"]

In [16]:
grp = test.groupby("verb")

In [17]:
verb_counts = grp.size().reset_index(name='obs').sort_values(['obs'], ascending=False)

In [18]:
verb_counts.iloc[:30]

,verb,obs
113,had,44
241,said,25
109,going,24
110,got,17
118,has,17
98,found,16
324,wanted,16
304,took,14
120,have,14
58,did,13
